# Day 18

## Part I

Ok，本日考的是Token解析和栈处理。首先需要解析整个表达式，这里有三种情况，可能是数字，可能是运算符加和乘，还可能是左右括号。还是为了代码清晰易读，用了枚举来代表解析出来的Token的类型，以及后续运算的状态，此处运算的状态只有两种，加法和乘法。

多说一句，在Python中写枚举的体验比起Rust差的太远，没法携带值，导致后续读取解析token的函数需要返回一个三元组，心塞。

In [1]:
from enum import Enum

TokenType = Enum('TokenType', ('digit', 'operator', 'parentheses'))

CalculateMode = Enum('CalculateMode', ('add', 'multiply'))

下面是第一个考点了，解析整个表达式。逐个字符读取分析，左括号、加号和乘号都很简单，直接返回相应的符号、类型和剩下的字符串即可；空格有两种情况，前面是数字或者前面是运算符，分别返回数字和忽略继续；虽然输入中的数字都是个位数，此处为了通用，还是使用了一个缓冲区用来存储数字，用来作为数字返回值；最后是右括号，分两种情况，如果前面是数字，就先返回数字，然后解析的指针不向前易懂，再次解析的时候在返回右括号：

In [2]:
from typing import Any, Tuple
import string

def next_token(data: str) -> Tuple[Any, TokenType, str]:
    data = data.rstrip()
    buf = ''
    for i, c in enumerate(data):
        if c == '(':
            return c, TokenType.parentheses, data[i+1:]
        if c == ')':
            if buf:
                return int(buf), TokenType.digit, data[i:]
            else:
                return c, TokenType.parentheses, data[i+1:]
        if c == '+' or c == '*':
            return c, TokenType.operator, data[i+1:]
        if c == ' ':
            if buf:
                return int(buf), TokenType.digit, data[i+1:]
            else:
                continue
        if c in string.digits:
            buf += c
    if buf:
        return int(buf), TokenType.digit, ''

用一个小测试看看解析的效果：

In [3]:
d = '1 + (2 * 3) + (4 * (5 + 6))'
while d:
    t, ttype, remain = next_token(d)
    print(f'{t:<2}, {ttype:^40} {remain}')
    d = remain

1 ,             TokenType.digit              + (2 * 3) + (4 * (5 + 6))
+ ,            TokenType.operator             (2 * 3) + (4 * (5 + 6))
( ,          TokenType.parentheses           2 * 3) + (4 * (5 + 6))
2 ,             TokenType.digit              * 3) + (4 * (5 + 6))
* ,            TokenType.operator             3) + (4 * (5 + 6))
3 ,             TokenType.digit              ) + (4 * (5 + 6))
) ,          TokenType.parentheses            + (4 * (5 + 6))
+ ,            TokenType.operator             (4 * (5 + 6))
( ,          TokenType.parentheses           4 * (5 + 6))
4 ,             TokenType.digit              * (5 + 6))
* ,            TokenType.operator             (5 + 6))
( ,          TokenType.parentheses           5 + 6))
5 ,             TokenType.digit              + 6))
+ ,            TokenType.operator             6))
6 ,             TokenType.digit              ))
) ,          TokenType.parentheses           )
) ,          TokenType.parentheses           


看起来不错，下面完成第一部分问题的一条表达式的计算逻辑。这里使用递归来计算括号中的部分，当然也可以使用栈来进行计算，但是显然递归的逻辑更加简单：

In [4]:
def part1_do_math(data: str) -> Tuple[int, str]:
    result = 0
    # 初始计算状态，加法
    mode = CalculateMode.add
    while data:
        token, token_type, remain = next_token(data)
        
        # 右括号，返回结果和剩余的字符串
        if token == ')':
            return result, remain
        
        # 左括号，递归计算括号中的部分，然后根据计算状态，与原来的临时结果相乘或相加
        if token == '(':
            ret, remain = part1_do_math(remain)
            if mode == CalculateMode.multiply:
                result *= ret
            else:
                result += ret
        
        # 数字，根据计算状态和临时结果相乘或相加
        if token_type == TokenType.digit:
            if mode == CalculateMode.multiply:
                result *= token
            else:
                result += token
        
        # 加号和乘号，修改计算状态
        if token == '+':
            mode = CalculateMode.add
        if token == '*':
            mode = CalculateMode.multiply
        
        # 使用剩余的字符串继续计算下面的部分
        data = remain

    return result

单元测试：

In [5]:
assert(part1_do_math('1 + 2 * 3 + 4 * 5 + 6') == 71)

assert(part1_do_math('1 + (2 * 3) + (4 * (5 + 6))') == 51)

assert(part1_do_math('2 * 3 + (4 * 5)') == 26)

assert(part1_do_math('5 + (8 * 3 + 9 + 3 * 4 * 3)') == 437)

assert(part1_do_math('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))') == 12240)

assert(part1_do_math('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2') == 13632)

读取输入文件函数：

In [6]:
from typing import List

def read_input(input_file: str) -> List[str]:
    with open(input_file) as fn:
        return fn.readlines()

第一部分计算结果的总和：

In [7]:
def part1_solution(datas: List[str]) -> int:
    return sum(part1_do_math(data) for data in datas)

获得第一部分结果：

In [8]:
datas = read_input('input.txt')
part1_solution(datas)

25190263477788

## Part II

第二部分需要再次使用一个栈算法，当然也可以使用递归实现，不过这里采取了自己使用list实现stack的方式。遇到乘法就将数据压栈，遇到加法就出栈然后进行加法计算后再次压栈，计算结果时只需要将栈中所有的数字相乘即可。这里使用numpy仅仅是为了直接使用prod ufuncs，使用reduce也是一样的：

In [9]:
import numpy as np

def do_math_part2(data: str) -> Tuple[int, str]:
    mode = CalculateMode.add
    stack = []
    while data:
        token, token_type, remain = next_token(data)
        
        # 右括号，返回栈内数字的乘积和剩余的字符串
        if token == ')':
            return np.prod(stack), remain
        
        # 左括号，递归计算结果，如果是乘法，将结果压栈，如果是加法，出栈与结果相加后再压栈
        if token == '(':
            ret, remain = do_math_part2(remain)
            if mode == CalculateMode.multiply:
                stack.append(ret)
            else:
                x = stack.pop() if stack else 0
                stack.append(x + ret)
        
        # 数字，乘法压栈，加法出栈与数字相加后再压栈
        if token_type == TokenType.digit:
            if mode == CalculateMode.multiply:
                stack.append(token)
            else:
                x = stack.pop() if stack else 0
                stack.append(x + token)
        
        # 设置计算状态
        if token == '+':
            mode = CalculateMode.add
        if token == '*':
            mode = CalculateMode.multiply
        data = remain
    return np.prod(stack)

单元测试：

In [11]:
assert(do_math_part2('1 + 2 * 3 + 4 * 5 + 6') == 231)

assert(do_math_part2('1 + (2 * 3) + (4 * (5 + 6))') == 51)

assert(do_math_part2('2 * 3 + (4 * 5)') == 46)

assert(do_math_part2('5 + (8 * 3 + 9 + 3 * 4 * 3)') == 1445)

assert(do_math_part2('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))') == 669060)

assert(do_math_part2('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2') == 23340)

计算第二部分问题表达式结果之和：

In [12]:
def part2_solution(datas: List[str]) -> int:
    return sum(do_math_part2(data) for data in datas)

获得第二部分结果：

In [13]:
part2_solution(datas)

297139939002972